## Example Experiment: Real-time curve fitting

This script takes 20 random bias measurements at different random setpoints and fits a curve (in real-time) to the relationship between voltage and current. It then applies calculated voltages to reach certain currents and prints the goal and measured results.


In [ ]:
# Import our libraries
import impulsePy as impulse
import numpy as np
from random import randrange
from time import sleep
import matplotlib.pyplot as plt
from scipy import optimize

# The line below opens the graph in a separate window (required for live updating)
%matplotlib qt

# Subscribe to data and event streams
impulse.bias.data.subscribe()
impulse.gas.data.subscribe()

impulse.waitForControl() # Wait until impulse is in control mode
sleep(2) # Wait for some data to come in

In [ ]:
#Voltage versus Current
def xparameter(dataFrame):
    xparameter = dataFrame["voltageMeasured"]
    return xparameter

def yparameter(dataFrame):
    yparameter = dataFrame["currentMeasured"]
    return yparameter

xparametername = "voltageMeasured (V)"
yparametername = "currentMeasued (A)"

def curveFunc(x,a,b,c):
    return (a*x*x) + (b*x) + c

# Create graph window
fig = plt.figure()
fig.canvas.set_window_title('Curve fitting')
ax = fig.add_subplot(111)
ax.set_xlabel(xparametername)
ax.set_ylabel(yparametername)
plt.ion()
plt.show()

xdat = []
ydat = []
xcrv = []
ycrv = []

biasparams = None

sc, = ax.plot(xdat,ydat, 'ro', label='Measurements')
crv, = ax.plot(xcrv, ycrv, label='Fitted curve')

impulse.bias.data.getNewData()
impulse.bias.data.setFlag("starting")

for i in range(20):
    impulse.bias.set(randrange(0,10,1),0.1)
    sleep(0.1)
    measurement = impulse.bias.data.getNewData()
    xdat.append(xparameter(measurement))
    ydat.append(yparameter(measurement))
    sc.set_ydata(ydat)
    sc.set_xdata(xdat)
    if i>2:
        dataF = impulse.bias.data.getDataFrame("starting")
        xPars = xparameter(dataF)
        yPars = yparameter(dataF)
        curvex = np.linspace(xPars.min(),xPars.max(),10)
        biasparams, biasparamsCovariance = optimize.curve_fit(curveFunc,xPars,yPars)
        crv.set_ydata(curveFunc(curvex,biasparams[0], biasparams[1], biasparams[2]))
        crv.set_xdata(curvex)   
    ax.relim()
    ax.autoscale_view()
    plt.legend(loc='best')    
    plt.pause(0.01)

currentsToCheck = [0.0005, 0.004]

def curveFunca(x, target):
    return (biasparams[0]*x[0]*x[0]) + (biasparams[1]*x[0]) + biasparams[2]-target

for currentCheck in currentsToCheck:
    setpoint = optimize.fsolve(curveFunc, [0.0], args=(biasparams[0],biasparams[1],biasparams[2]-currentCheck))
    impulse.bias.set(setpoint[0],0.1)
    print(f"Current goal: {currentCheck}, voltage set: {setpoint[0]}")
    sleep(0.2)
    currentMeas=impulse.bias.data.getNewData()["currentMeasured"]
    print(f"Measured Current: {currentMeas}")
    sleep(1)


In [ ]:
impulse.disconnect()